Cellule 1 - Configuration et chemins

In [ ]:
from pathlib import Path
import pandas as pd
import pdfplumber
from tqdm.notebook import tqdm # Pour une jolie barre de progression

# Configuration des chemins relative au notebook
# (Si le notebook est dans /notebooks, on remonte d'un cran pour la racine)
PROJECT_ROOT = Path.cwd().parent
PDF_DIR = PROJECT_ROOT / "data" / "synthetic" / "pdfs"
OUT_CSV = PROJECT_ROOT / "data" / "synthetic" / "pdf_texts.csv"

# Vérification immédiate
if not PDF_DIR.exists():
    print(f"❌ Erreur : Dossier introuvable -> {PDF_DIR}")
else:
    print(f"✅ Dossier prêt : {len(list(PDF_DIR.glob('*.pdf')))} fichiers trouvés.")

Cellule 2 - Fonctions utilitaires

In [ ]:
def extract_text_from_pdf(pdf_path: Path) -> str:
    """Ouvre un PDF et extrait le texte brut de toutes les pages."""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            # On concatène le texte ou une chaîne vide si l'extraction échoue sur une page
            content = "\n".join([page.extract_text() or "" for page in pdf.pages])
            return content.strip()
    except Exception as e:
        print(f"⚠️ Erreur sur {pdf_path.name}: {e}")
        return ""

Cellule 3 - Exécution

In [ ]:
pdf_files = sorted(PDF_DIR.glob("*.pdf"))
data = []

print("🚀 Lancement de l'extraction...")

for f in tqdm(pdf_files, desc="Extraction des PDF"):
    text_content = extract_text_from_pdf(f)
    data.append({
        "pdf_id": f.stem,
        "file_name": f.name,
        "text": text_content,
        "text_len": len(text_content)
    })

# Création du DataFrame
df = pd.DataFrame(data)

Cellule 4 - Sauvegarde et aperçu

In [ ]:
# Sauvegarde
df.to_csv(OUT_CSV, index=False, encoding="utf-8")
print(f"💾 Dataset sauvegardé : {OUT_CSV}")

# Aperçu rapide des stats
print("\n--- Statistiques ---")
print(df["text_len"].describe())

# Affichage des 5 premières lignes
df.head()